# 2-D Ising model part two (exercise)

![Example output](images/ising_animation.gif)

The previous notebook ended with a two-array implementation that avoided race-conditions but was no longer faithfully modeling the physics of the system.

We cannot just compute the new spins by looking at the old state. But how do we compute new spin values by respecting the data dependencies in the sequential Metropolis algorithm?

We will need to **rethink the algorithm** for correct parallel execution.

## Using a Checkerboard update

This is a common technique for problems with local dependencies on a grid.

The algorithm gets its name because we conceptually divide the lattice sites into two groups, like the
black and white squares of a checkerboard.

**Crucially, spins on black squares only have neighbors on white squares, and spins on white squares only have neighbors on black squares.**

Instead of trying to update all lattice sites in one go (which, as we saw with the naive parallel approach, leads to problems because spin updates wouldn't see each other's changes within the same pass), we perform two distinct update steps per iteration:

1. **Update all "black" sites in parallel.** They read the current state of their "white" neighbors.
2. **Update all "white" sites in parallel.** They read the newly updated state of their "black" neighbors from Step 1.

Our earlier naive parallel approach was flawed because spins updated simultaneously couldn't see each other's changes from that same pass. This is different from the sequential algorithm.

By only updating one checkerboard population at a time, we ensure that the neighbor states of each lattice site are never out-of-sync.

The two-step process ensures that when any spin is being updated, the neighbor values it reads are consistent and appropriate for that stage of the parallel computation.

![Checkerboard labelling](images/checkerboard_figure.png)

*Image credit: [Romero et al. (2019)](https://arxiv.org/abs/1906.06297)*

Now, we need to allocate separate arrays for the black and white populations.

They will be of size $N$ by $N/2$. We will assume $N$ is even.

We will also allocate a $N$ by $N$ array for the combined lattice. This is not needed for the computation, but
it does help with visualization.

In [ ]:
import warp as wp

LATTICE_SIZE = 256

lattice_b = wp.empty((LATTICE_SIZE, LATTICE_SIZE // 2), dtype=wp.int8)
lattice_w = wp.empty_like(lattice_b)

# For plotting
combined_lattice = wp.empty((LATTICE_SIZE, LATTICE_SIZE), dtype=wp.int8)


# We can continue to use the following kernel from the previous notebook
@wp.kernel
def generate_lattice(lattice: wp.array2d(dtype=wp.int8), rng_seed: int):
    i, j = wp.tid()

    # Generate random number between [0.0, 1.0).
    # NOTE: To get different values, we can change either (or both) arguments
    rng_state = wp.rand_init(rng_seed, i * lattice.shape[1] + j)
    if wp.randf(rng_state, 0.0, 1.0) < 0.5:
        lattice[i, j] = wp.int8(1)
    else:
        lattice[i, j] = wp.int8(-1)

We can use the following kernel to create the combined lattice from `lattice_b` and `lattice_w`:

In [2]:
@wp.kernel
def combine_lattices(
    lattice_b: wp.array2d(dtype=wp.int8),
    lattice_w: wp.array2d(dtype=wp.int8),
    combined_lattice: wp.array2d(dtype=wp.int8),
):
    i, j = wp.tid()

    if i % 2 == 0:
        combined_lattice[i, 2 * j] = lattice_w[i, j]
        combined_lattice[i, 2 * j + 1] = lattice_b[i, j]
    else:
        combined_lattice[i, 2 * j] = lattice_b[i, j]
        combined_lattice[i, 2 * j + 1] = lattice_w[i, j]

Let's how it works. We'll set all values of `lattice_w` to 1 and all values of `lattice_b` to 0, and then we'll combine the two arrays by running the `combine_lattices` kernel.

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt

lattice_w.fill_(1)
lattice_b.fill_(-1)

wp.launch(combine_lattices, dim=lattice_b.shape, inputs=[lattice_b, lattice_w, combined_lattice])

fig = plt.figure()
plt.imshow(combined_lattice.numpy(), cmap="viridis")
plt.title(f"Lattice {LATTICE_SIZE}x{LATTICE_SIZE}")
plt.show()

Next, we will write a version of the `update_lattice` kernel that implements the checkerboard algorithm.

It will take the following inputs:

- `beta: float` (Same as before)
- `rng_seed: int` (Same as before)
- `is_black: bool` Tells us whether `lattice` is for the black or white conceptual lattice groups
- `op_lattice: wp.array2d(dtype=wp.int8)` The lattice of the opposite color.
- `lattice: wp.array2d(dtype=wp.int8)` The lattice of the color being updated.

We always find the "first" horizontal neighbor at the position `op_lattice[i, j]`.

The neighbor indexing logic for the "second" horizontal neighbor becomes a little complicated, but can be reasoned by considering the four situations:

- `is_black` is `True`, `i` is even
- `is_black` is `True`, `i` is even
- `is_black` is `False`, `i` is odd
- `is_black` is `False`, `i` is odd

In each of the four above cases, the column at which we find the second horizontal neighbor in `op_lattice` is different.

In [4]:
@wp.kernel
def update_lattice(
    beta: float,
    rng_seed: int,
    is_black: bool,
    op_lattice: wp.array2d(dtype=wp.int8),
    lattice: wp.array2d(dtype=wp.int8),
):
    i, j = wp.tid()

    lattice_size = lattice.shape[0]

    num_sub_cols = lattice_size // 2

    # For sub-lattice columns (used for horizontal neighbors)
    sub_col_left_idx = (j - 1 + num_sub_cols) % num_sub_cols
    sub_col_right_idx = (j + 1) % num_sub_cols

    # Determine the sub-lattice column index (in op_lattice) for the "second" horizontal neighbor.
    # op_lattice[i, j] will always be the "first" horizontal neighbor due to the checkerboard mapping.

    if is_black:
        # Current spin is B[i, j]
        # op_lattice is W
        if i % 2 == 0:
            # B is at global (i, 2*j + 1)
            # Horizontal W neighbors are W[i, j] (global left)
            # and W[i, sub_col_right_idx] (global right)
            second_horizontal_neighbor_sub_col_idx = sub_col_right_idx
        else:
            # B is at global (i, 2*j)
            # Horizontal W neighbors are W[i, sub_col_left_idx] (global left)
            # and W[i, j] (global right)
            second_horizontal_neighbor_sub_col_idx = sub_col_left_idx
    else:
        # Current spin is W[i, j]
        # op_lattice is B
        if i % 2 == 0:
            # W is at global (i, 2*j)
            # Horizontal B neighbors are B[i, sub_col_left_idx] (global left)
            # and B[i, j] (global right)
            second_horizontal_neighbor_sub_col_idx = sub_col_left_idx
        else:
            # W is at global (i, 2*j + 1)
            # Horizontal B neighbors are B[i, j] (global left)
            # and B[i, sub_col_right_idx] (global right)
            second_horizontal_neighbor_sub_col_idx = sub_col_right_idx

    # Neighbors: top, bottom, horizontal 1, horizontal 2
    nn_sum = (
        op_lattice[(i - 1 + lattice_size) % lattice_size, j]
        + op_lattice[(i + 1) % lattice_size, j]
        + op_lattice[i, j]
        + op_lattice[i, second_horizontal_neighbor_sub_col_idx]
    )

    # Determine whether to flip spin
    spin_ij = lattice[i, j]
    acceptance_ratio = wp.exp(-2.0 * beta * wp.float32(nn_sum) * wp.float32(spin_ij))

    # Generate random number between [0.0, 1.0).
    # Use num_sub_cols for consistency in unique seed generation
    rng_state = wp.rand_init(rng_seed, i * num_sub_cols + j)

    if wp.randf(rng_state, 0.0, 1.0) < acceptance_ratio:
        lattice[i, j] = -spin_ij


In [ ]:
import numpy as np
import IPython.display
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

# --- Simulation Parameters ---
TEMPERATURE = 2.269  # Critical temperature for 2D Ising is ~2.269 (J=1, k_B=1)
# Try other temperatures: T_low = 1.0 (ordered), T_high = 5.0 (disordered)
BETA = 1.0 / TEMPERATURE
LATTICE_SIZE = 256

lattice_b = wp.empty((LATTICE_SIZE, LATTICE_SIZE // 2), dtype=wp.int8)
lattice_w = wp.empty_like(lattice_b)
# For plotting
combined_lattice = wp.empty((LATTICE_SIZE, LATTICE_SIZE), dtype=wp.int8)

# Reset initial conditions
wp.launch(generate_lattice, lattice_b.shape, inputs=[lattice_b, 17])
wp.launch(generate_lattice, lattice_w.shape, inputs=[lattice_w, 42])

# Get viridis colormap
viridis = plt.cm.viridis
norm = Normalize(vmin=-1, vmax=1)

# Collect frames as arrays instead of images
frames = []

for i in range(200):
    # Update black (is_black=True)
    wp.launch(
        update_lattice,
        (LATTICE_SIZE, LATTICE_SIZE // 2),
        inputs=[BETA, i + 17, True, lattice_w, lattice_b],
    )
    # Update white (is_black=False)
    wp.launch(
        update_lattice,
        (LATTICE_SIZE, LATTICE_SIZE // 2),
        inputs=[BETA, i + 42, False, lattice_b, lattice_w],
    )

    # Combine the lattices
    wp.launch(
        combine_lattices, dim=lattice_b.shape, inputs=[lattice_b, lattice_w, combined_lattice]
    )

    # Copy to CPU and apply colormap
    normalized_lattice = norm(combined_lattice.numpy())
    colored_frame = viridis(normalized_lattice)

    # Convert to RGB
    rgb_frame = (colored_frame[:, :, :3] * 255).astype(np.uint8)
    frames.append(rgb_frame)

# Convert to PIL images and save
pil_images = [Image.fromarray(frame, mode="RGB") for frame in frames]
output_filename = f"output/{LATTICE_SIZE}x{LATTICE_SIZE}_{TEMPERATURE}.gif"

pil_images[0].save(
    output_filename, save_all=True, append_images=pil_images[1:], duration=100, loop=0
)

IPython.display.Image(output_filename)

This looks more correct than our previous attempts, but how do we know it's actually working as intended?

Let's write a kernel to compute the total magnetization so that we can look at the time evolution of the quantity.

Since we aren't building the combined lattice on every iteration, we will need to have it take the `lattice_w` and `lattice_b` arrays as inputs.

In [6]:
@wp.kernel
def compute_total_magnetization(
    lattice_b: wp.array2d(dtype=wp.int8),
    lattice_w: wp.array2d(dtype=wp.int8),
    result: wp.array(dtype=wp.int32),
):
    i, j = wp.tid()

    local_val = wp.int32(lattice_b[i, j] + lattice_w[i, j])
    wp.atomic_add(result, 0, local_val)

In [ ]:
# --- Simulation Parameters ---
TEMPERATURE = 2.269  # Critical temperature for 2D Ising is ~2.269 (J=1, k_B=1)
# Try other temperatures: T_low = 1.0 (ordered), T_high = 5.0 (disordered)
BETA = 1.0 / TEMPERATURE
TOTAL_MCS = 1000

# Also allocate memory for total magnetization
magnetization_values = []
total_magnetization = wp.zeros(1, dtype=wp.int32)

# Reset initial conditions
wp.launch(generate_lattice, lattice_b.shape, inputs=[lattice_b, 17])
wp.launch(generate_lattice, lattice_w.shape, inputs=[lattice_w, 42])


for step in range(TOTAL_MCS):
    # Update black (is_black=True)
    wp.launch(
        update_lattice,
        (LATTICE_SIZE, LATTICE_SIZE // 2),
        inputs=[BETA, step + 17, True, lattice_w, lattice_b],
    )
    # Update white (is_black=False)
    wp.launch(
        update_lattice,
        (LATTICE_SIZE, LATTICE_SIZE // 2),
        inputs=[BETA, step + 42, False, lattice_b, lattice_w],
    )
    # Compute total magnetization
    total_magnetization.zero_()
    wp.launch(
        compute_total_magnetization,
        lattice_b.shape,
        inputs=[lattice_b, lattice_w, total_magnetization],
    )

    # Append average magnetization to a Python list
    magnetization_values.append(total_magnetization.numpy()[0] / (LATTICE_SIZE * LATTICE_SIZE))

# Plot result
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(magnetization_values, "-", linewidth=1.5, color="#76b900")

ax.set_xlabel("Monte Carlo Steps", fontsize=12)
ax.set_ylabel("Magnetization (m)", fontsize=12)
ax.set_title(f"Magnetization vs. Monte Carlo Steps (T={TEMPERATURE:.3f})", fontsize=14)
ax.grid(True, alpha=0.3)
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False

plt.tight_layout()

There should be something unexpected about the previous plot. As the temperature is reduced below the critical temperature, we should see spontaneous magnetization.

Try lowering the temperature and see what happens. Is there a bug in the code?

If you aren't sure what's wrong, a common debugging strategy is to run the script in debug mode by adding:

```python
import warp as wp

wp.config.mode = "debug"
```

In the next cell, we are running a standalone script in debug mode (due to some complications with how Warp runs in a Jupyter notebook, it isn't easy to turn on debug mode in the middle of a notebook).

In [ ]:
!python sources/ising-checkerboard-debug.py

## Comparison with analytical solution

Finally, we can write some code that plots the steady-state absolute value of magnetization across a range of temperatures.

For $T < T_{\mathrm{crit}}$, there is actually an analytic solution discovered by Onsager in 1944 for the spontaneous magnetization of a 2-D Ising model in zero external magnetic field for temperatures below the critical temperature.

\begin{align}
    M = \left[1 - \frac{1}{\left(\sinh \left( \frac{2 J}{k_B T} \right) \right)^4}\right]^{1/8}
\end{align}

For $T > T_c$, the system is in a paramagnetic state with no net magnetization ($M = 0$).

In [ ]:
import statistics
import numpy as np


def calculate_steady_state_magnetization(temperature):
    BETA = 1.0 / temperature

    # Also allocate memory for total magnetization
    magnetization_values = []
    total_magnetization = wp.zeros(1, dtype=wp.int32)

    # Reset initial conditions
    wp.launch(generate_lattice, lattice_b.shape, inputs=[lattice_b, 17])
    wp.launch(generate_lattice, lattice_w.shape, inputs=[lattice_w, 42])

    # Warmup
    for step in range(3000):
        wp.launch(
            update_lattice,
            (LATTICE_SIZE, LATTICE_SIZE // 2),
            inputs=[BETA, step + 17, True, lattice_w, lattice_b],
        )
        # Update white (is_black=False)
        wp.launch(
            update_lattice,
            (LATTICE_SIZE, LATTICE_SIZE // 2),
            inputs=[BETA, step + 42, False, lattice_b, lattice_w],
        )

    for step in range(1000):
        # Update black (is_black=True)
        wp.launch(
            update_lattice,
            (LATTICE_SIZE, LATTICE_SIZE // 2),
            inputs=[BETA, step + 17, True, lattice_w, lattice_b],
        )
        # Update white (is_black=False)
        wp.launch(
            update_lattice,
            (LATTICE_SIZE, LATTICE_SIZE // 2),
            inputs=[BETA, step + 42, False, lattice_b, lattice_w],
        )
        # Compute total magnetization
        total_magnetization.zero_()
        wp.launch(
            compute_total_magnetization,
            lattice_b.shape,
            inputs=[lattice_b, lattice_w, total_magnetization],
        )

        # Append average magnetization to a Python list
        magnetization_values.append(
            abs(total_magnetization.numpy()[0]) / (LATTICE_SIZE * LATTICE_SIZE)
        )

    return statistics.mean(magnetization_values), statistics.stdev(magnetization_values)


# Recall that the critical temperature is 2.269
temperatures = [1.4, 1.6, 1.8, 2.0, 2.1, 2.15, 2.2, 2.25, 2.3, 2.4, 2.5, 2.7, 3.0]
magnetization_values = []
magnetization_stdev_values = []
for temperature in temperatures:
    m, m_stdev = calculate_steady_state_magnetization(temperature)

    magnetization_values.append(m)
    magnetization_stdev_values.append(m_stdev)

# Plot result
fig, ax = plt.subplots(figsize=(10, 6))

plt.errorbar(
    temperatures,
    magnetization_values,
    yerr=magnetization_stdev_values,
    fmt="o--",
    capsize=5,
    capthick=2,
    label="Simulation ± σ",
    color="#76b900",
)

# Add Onsager solution
T_crit = 2 / np.log(1 + np.sqrt(2))
T_theory = np.linspace(temperatures[0], T_crit * 0.999, 500)
M_onsager = (1 - np.sinh(2 / T_theory) ** (-4)) ** (1 / 8)

plt.plot(T_theory, M_onsager, "-", linewidth=2, label="Onsager Solution", color="#7209B7")
plt.axvline(T_crit, color="black", linestyle="--", label=f"T_c = {T_crit:.3f}")
plt.legend()

ax.set_xlabel("Temperature (T)", fontsize=12)
ax.set_ylabel("Magnetization (M)", fontsize=12)
ax.set_title("Magnetization vs. Temperature", fontsize=14)
ax.grid(True, alpha=0.3)
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.resizable = False

plt.tight_layout()

## Summary

In this notebook, we examined different attempts to simulate the Ising model in two dimensions.

The main intention behind this exercise was to illuminate some considerations when writing algorithms that run **correctly** and **efficiently** on a GPU.

Starting from a sequential Python version, we saw issues with race conditions when attempting to update the lattice array in-place.

Moving to separate "current" and "updated" lattice arrays removes the race condition, but resulted in an algorithm that no longer exhibited the physics of the sequential algorithm.

Finally, we saw that a checkerboard algorithm allows us to recover the intended physics while addressing the data dependency issues that had hindered the initial approaches.
